In [1]:
import numpy as np
import cv2
import sys
import glob
import sys, os

#from skimage.morphology import skeletonize
from skimage import img_as_ubyte
from skimage.util import invert
from skimage import morphology, filters
import matplotlib.pyplot as plt
from random import randint
import time
import pandas as pd
import re
import collections

In [2]:
#declaration of global variables
#dataset path
pathDS = "../dataset/"

#initialize skeleton models and variables for individual frame skeleton detection
protoFile = "../MPI/pose_deploy_linevec.prototxt"
weightsFile = "../MPI/pose_iter_160000.caffemodel"
#load pretrained models for single person joint detection
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
jointCount0 = 15
POSE_PAIRS =[ [1,0],[1,2],[1,5],[2,3],[3,4],[5,6],[6,7],[1,8],[8,9],[9,10],[1,11],[11,12],[12,13],[0,14],[0,15],[14,16],[15,17]]

#directories list of dataset for each class
category = [cat for cat in os.listdir('../dataset')]
'''mutliple dancers joint keypoint extraction weight and network model trained on coco'''

#load weight and network files
cocoProtoFile = "../coco/pose_deploy_linevec.prototxt"
cocoCaffeWeightsFile = "../coco/pose_iter_440000.caffemodel"
jointCount = 18

#initialize skeleton pair nodes and labels based on coco training dataset
keypointPairs = [[1,2], [1,5], [2,3], [3,4], [5,6], [6,7],[1,8], [8,9], [9,10], [1,11], [11,12], [12,13],[1,0], [0,14], [14,16], [0,15], [15,17],[2,17], [5,16] ]
keypointsMapping = ['Nose', 'Neck', 'R-Sho', 'R-Elb', 'R-Wr', 'L-Sho', 'L-Elb', 'L-Wr', 'R-Hip', 'R-Knee', 'R-Ank', 'L-Hip','L-Knee', 'L-Ank', 'R-Eye', 'L-Eye', 'R-Ear', 'L-Ear']                  
keypointPairsIndex = [[31,32], [39,40], [33,34], [35,36], [41,42], [43,44],[19,20], [21,22], [23,24], [25,26], [27,28], [29,30], [47,48], [49,50], [53,54], [51,52], [55,56], [37,38], [45,46]]
keypointPairsColor = [ [0,100,255], [0,100,255], [0,255,255], [0,100,255], [0,255,255], [0,100,255],[0,255,0], [255,200,100], [255,0,255], [0,255,0], [255,200,100], [255,0,255],[0,0,255], [255,0,0], [200,200,0], [255,0,0], [200,200,0], [0,0,0]]
#initialize the pre-trained coco network using cv dnn, weight from caffe file, network from protofile
cocoDnn = cv2.dnn.readNetFromCaffe(cocoProtoFile, cocoCaffeWeightsFile)

In [3]:
clips = [clip for clip in os.listdir('../dataset/shoa/newd/')]
saveDir = '../dataset/shoa/newd/frames/'


def extractFrameData():
    i = 0
    for clip in clips:
        
        if str(clip) == "frames":
            continue
        
        print (clip)
        img =cv2.VideoCapture("../dataset/shoa/newd/" + str(clip))
        path = saveDir + "clip" + str(i) + "/"
        
        isExist = os.path.exists(path)
        
        if(not isExist):
            
            os.makedirs(path)
    
        i+=1
        extractClipFrame(img,path)
            
        
        
def extractClipFrame(clip,path):
    
    j = 0
    while True:
        # read video frames
        retval, frame = clip.read()
        # check whether the frames have been grabbed
        if not retval:
            return
        # resize video frames
        frame = cv2.resize(frame, (640, 360))

        #skeletonizeDancePose(frame,0.1)    
        cv2.imwrite(path + "/" + str(j)+ '.jpg',frame)
        j = j + 1

In [4]:
extractFrameData()

3-4-7.mp4
6-400-403.mkv
4-10-13.mkv
3-29-31.mp4
4-13-16.mkv
2-30-50.mkv
3-88-90.mp4
3-200-201.mp4
3-247-250.mp4
6-110-112.mkv
5-380-387.mkv
4-129-136.mkv
5-314-334.mkv
5-361-371.mkv


GENERAL HELPER FUNCTIONS

In [3]:
#hlper function id:HELPER 0.1
#for sorting files in dir 
def checkFileString(fileString):
    return int(fileString) if fileString.isdigit() else fileString
def natural_keys(fileList):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ checkFileString(i) for i in re.split(r'(\d+)', fileList) ]

#hlper function id:HELPER 0.2
#sort dancer position based on neck x coordinate left to right
def sortDancerPosition(data):
    #initialize imp vals
    data2 = data.copy()
    index = 0
    #zeros numpy array to hold the sorted data
    orderedDancers = np.zeros((data2.shape[0], 18,2))
    
    #get the target x coordinate float value and it's index
    indexVal = [i for i in range(data2.shape[0])]
    floatVal = []
    for j in range(data2.shape[0]):
        floatVal.append(data[j][1][0])

    valIndexPair = dict(zip(floatVal, indexVal))
    oValIndexPair = collections.OrderedDict(sorted(valIndexPair.items()))
    #sort the dictionary based on x float value
    oValIndexPair = collections.OrderedDict(sorted(valIndexPair.items()))
    
    orderedDancers = np.zeros((data2.shape[0], 18,2))
    index = 0
    for k, v in oValIndexPair.items():
        orderedDancers[index] = data[v]
        index+=1
        
    return orderedDancers

PROBLEM SPECIFIC HELPER FUNCTION

HELPER FUNCTION: HELPER 1.0
SINGLE PERSON SKELETON EXTRACTOR FUNCTION

In [14]:
#for single person skeleton extraction
def skeletonizeDancePose(fileName, thresh):
    
    img = cv2.imread(fileName)
    frameWidth = img.shape[1]
    frameHeight = img.shape[0]
    frameCopy = np.copy(img)
    threshold = 0.1
    
    #preprocess img for input
    inpBlob = cv2.dnn.blobFromImage(img, 1.0 / 255, (368, 368),(0, 0, 0), swapRB=False, crop=False)
    #feed the preprocessed img through the network
    cocoDnn.setInput(inpBlob)
    output = cocoDnn.forward()
    H = output.shape[2]
    W = output.shape[3]
    
    # gather keypoints here
    points = []
    
    #loop through possible 18 keypoints
    for i in range(jointCount):
        # confidence map of corresponding body's part.
        probMap = output[0, i, :, :]

        # Find global maxima of the probMap.
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

        # Scale the point to fit on the original image
        x = (frameWidth * point[0]) / W
        y = (frameHeight * point[1]) / H

        if prob > thresh :
            cv2.circle(frameCopy, (int(x), int(y)), 8, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frameCopy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)
            cv2.circle(img, (int(x), int(y)), 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)

            # Add the point to the list if the probability is greater than the threshold
            points.append((int(x), int(y)))
        else :
            points.append((0,0))
        
    # Draw Skeleton
    for pair in POSE_PAIRS:
        partA = pair[0]
        partB = pair[1]

        if points[partA] and points[partB]:
            cv2.line(img, points[partA], points[partB], (0, 255, 255), 3)
    
    return np.array(points)
    plt.figure(figsize=[10,10])
    plt.imshow(cv2.cvtColor(frameCopy, cv2.COLOR_BGR2RGB))
    plt.figure(figsize=[10,10])
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    #cv2.imshow("Subtraction result", frame)
    
    #return the curent 18 body part point pose
    

HELPER FUNCTION: HELPER 2.0
MULTI PERSON SKELETON EXTRACTOR FUNCTION

In [15]:
def extractKeypoints(fileName):
    #initialize keypoint pairs and keypoint holders
    validKeypointPairs = []
    invalidKeypointPairs = []
    detectedKeypoints = []
    keypointsList = np.zeros((0,3))
    keypointId = 0
    threshold = 0.1
    dancerKeypoints = -1 * np.ones((0, 19))
    #initialize threshhold metrices for part affinities of joint flow and keypoints and interopolation points
    interpSamples = 10
    pafThresh = 0.1
    keypointThresh = 0.7
    #open frame and extract frame properties
    img = cv2.imread(fileName)
    imgWidth = img.shape[1]
    imgHeight = img.shape[0]
    
    t = time.time()
    
    #use CPU for dnn prediction
    cocoDnn.setPreferableBackend(cv2.dnn.DNN_TARGET_CPU)
    #prepare frame for keyframe estemation
    inputH,inputW =368,int((368/imgHeight)*imgWidth)
    #extract blobs from frame
    connectedPixelPairs = cv2.dnn.blobFromImage(img, 1.0 / 255, (inputW, inputH),(0, 0, 0), swapRB=False, crop=False)
    #feed blob to the coco network
    cocoDnn.setInput(connectedPixelPairs)
    output = cocoDnn.forward()
    #detect keypoints
    for joint in range(jointCount):
        probMap = output[0,joint,:,:]
        probMap = cv2.resize(probMap, (img.shape[1], img.shape[0]))
        #extract keypoints using probabiliy map   
        probMap2 = cv2.GaussianBlur(probMap,(3,3),0,0)
        probMapMask = np.uint8(probMap2>threshold)
        keypoints = []
        #find blobs
        mapContours, _ = cv2.findContours(probMapMask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        #for each blob find the maxima
        for c in mapContours:
            cMask = np.zeros(probMap2.shape)
            cMask = cv2.fillConvexPoly(cMask, c, 1)
            cMapMaskProb = probMap2 * cMask
            _, maxVal, _, maxLoc = cv2.minMaxLoc(cMapMaskProb)
            keypoints.append(maxLoc + (probMap[maxLoc[1], maxLoc[0]],))
        
        #print("Keypoints - {} : {}".format(keypointsMapping[joint], keypoints))
        keypointIds = []
        for i in range(len(keypoints)):
            keypointIds.append(keypoints[i] + (keypointId,))
            keypointsList = np.vstack([keypointsList, keypoints[i]])
            keypointId += 1

        detectedKeypoints.append(keypointIds)
    
    #loop through keypoint paris
    for k in range(len(keypointPairsIndex)):
        #on each iteration create keypoint(body part) pairs
        paf1 = output[0, keypointPairsIndex[k][0], :, :]
        paf2 = output[0, keypointPairsIndex[k][1], :, :]
        #resize to inputframe
        paf1 = cv2.resize(paf1, (imgWidth, imgHeight))
        paf2 = cv2.resize(paf2, (imgWidth, imgHeight))
        # Find the keypoints for the current body part pairs
        cand1 = detectedKeypoints[keypointPairs[k][0]]
        cand2 = detectedKeypoints[keypointPairs[k][1]]
        n1 = len(cand1)
        n2 = len(cand2)
        # If keypoints for the joint-pair is detected
        # check every joint in cand1 with every joint in cand2
        # Calculate the distance vector between the two joints
        # Find the PAF values at a set of interpolated points between the joints
        if( n1 != 0 and n2 != 0):
            validKeypointPair = np.zeros((0,3))
            for i in range(n1):
                maxP=-1
                maxScore = -1
                found = 0
                for j in range(n2):
                    # Find a good pair vector points
                    goodPair = np.subtract(cand2[j][:2], cand1[i][:2])
                    norm = np.linalg.norm(goodPair)
                    if norm:
                        goodPair = goodPair / norm
                    else:
                        continue
                    # Find interpolated point between two joints
                    interpXY = list(zip(np.linspace(cand1[i][0], cand2[j][0], num=interpSamples),
                                            np.linspace(cand1[i][1], cand2[j][1], num=interpSamples)))
                    # find paf from interPXY
                    interpPaf = []
                    for k in range(len(interpXY)):
                        interpPaf.append([paf1[int(round(interpXY[k][1])), int(round(interpXY[k][0]))],
                                           paf2[int(round(interpXY[k][1])), int(round(interpXY[k][0]))] ]) 
                    # Find E
                    pafScores = np.dot(interpPaf, goodPair)
                    avgPafScores = sum(pafScores)/len(pafScores)
                    
                    # Check if the connection is valid
                    # If the fraction of interpolated vectors aligned with PAF is higher then threshold it's a Valid Pair  
                    if (len(np.where(pafScores > pafThresh)[0]) / interpSamples ) > keypointThresh :
                        if avgPafScores > maxScore:
                            maxP = j
                            maxScore = avgPafScores
                            found = 1
                # Append the connection to the list
                if found:            
                    validKeypointPair = np.append(validKeypointPair, [[cand1[i][3], cand2[maxP][3], maxScore]], axis=0)

            # Append the detected connections to the global list
            validKeypointPairs.append(validKeypointPair)
        else: # If no keypoints are detected
            #print("No Connection : k = {}".format(k))
            invalidKeypointPairs.append(k)
            validKeypointPairs.append([])
    #get separate points for each dancers pose
    for k in range(len(keypointPairsIndex)):
        if k not in invalidKeypointPairs:
            jointPart1 = validKeypointPairs[k][:,0]
            jointPart2 = validKeypointPairs[k][:,1]
            indexVal1, indexVal2 = np.array(keypointPairs[k])

            for i in range(len(validKeypointPairs[k])): 
                found = 0
                dancerIndex = -1
                for j in range(len(dancerKeypoints)):
                    if dancerKeypoints[j][indexVal1] == jointPart1[i]:
                        dancerIndex = j
                        found = 1
                        break
                if found:
                    dancerKeypoints[dancerIndex][indexVal2] = jointPart2[i]
                    dancerKeypoints[dancerIndex][-1] += keypointsList[jointPart2[i].astype(int), 2] + validKeypointPairs[k][i][2]
                # if find no partA in the subset, create a new subset
                elif not found and k < 17:
                    row = -1 * np.ones(19)
                    row[indexVal1] = jointPart1[i]
                    row[indexVal2] = jointPart2[i]
                    # add the keypoint_scores for the two keypoints and the paf_score 
                    row[-1] = sum(keypointsList[validKeypointPairs[k][i,:2].astype(int), 2]) + validKeypointPairs[k][i][2]
                    dancerKeypoints = np.vstack([dancerKeypoints, row])
    #result visualization
    img2 = img.copy()
    
    #ordered keypoint and draw keypoint on image for visualization
    orderedKeypoint = np.zeros((len(dancerKeypoints), 18,2))
    
    for i in range(17):
        for n in range(len(dancerKeypoints)):
            index = dancerKeypoints[n][np.array(keypointPairs[i])]
            
            for jp in index:
                if(jp != -1):    
                    jointPoint = np.int32(keypointsList[jp.astype(int)])
                    
                    #print(jointPoint[:2])
                    
                    orderedKeypoint[n][keypointPairs[i][np.where(index == jp)[0][0]]]=jointPoint[:2]
            
            if -1 in index:     
                continue
             
            B = np.int32(keypointsList[index.astype(int), 0])
            A = np.int32(keypointsList[index.astype(int), 1])

            cv2.line(img2,(B[1], A[1]),(B[0], A[0]), keypointPairsColor[i], 3, cv2.LINE_AA)
    
    return(img2,orderedKeypoint)
    plt.figure(figsize=[15,15])
    plt.imshow(img2[:,:,[2,1,0]])
    
    
    
    

HELPER FUNCTION: HELPER 3.0
FUNCTION TO EXTRACT CLIP FRAMES AS AN IN PUT FOR MULTI PERSON SKELETON EXTRACTOR HELPER FUNCTION: HELPER 2.0

In [19]:
#helper function to go through frame sequence in the dataset folder
def passFrameSequence(multiPerson):
    
    modePathString = "indvg"
    if(multiPerson):
        
        modePathString = "frames"
    #class labels
    classLabels = ['guraginga','gojjam',"shoa"]
    frametracker = 0
    clipTracker = 0
    
    for label in classLabels:
        #currentFramePath
        currentDSFramePath = os.path.join('../dataset', label + "/" + modePathString)
        currentClipPath = [i for i in os.listdir(currentDSFramePath) if not i.startswith('.')]
        currentClipPath.sort(key=natural_keys)
        
        #loop through each clips
        for clip in currentClipPath:
            
            currentFramePath = [j for j in os.listdir(os.path.join(currentDSFramePath,clip)) if not j.startswith('.')]
            currentFramePath.sort(key=natural_keys)
            
            timeStep = 0
            dancerLen = 0
            
            curentFrameSequenceKeypointList = []
            for frame in currentFramePath:
                
                if(timeStep==5): continue
                fullPath = os.path.join(currentDSFramePath, clip)
                frameName = os.path.join(fullPath, frame)
                
                if(not multiPerson):
                    currentKeypoints = skeletonizeDancePose(frameName,0.1)
                    curentFrameSequenceKeypointList.append(currentKeypoints)
                    print("****** current frame being processed: ",frame + " ******\n")
                    timeStep+=1
                    continue
                
                data = extractKeypoints(frameName) 
                
                currentKeypoints = sortDancerPosition(data[1])
                skeletonImg = data[0]

                
                if(timeStep == 0):
                    dancerLen = currentKeypoints.shape[0]
                    timeStep+=1
                
                print("****** current frame being processed: ",frame + " ******\n")
                print("****** Current dancers Numbers " + str(currentKeypoints.shape[0]) + " ******")  
                #check if dancer number is not curroptued
                print( "****** Prev dancers number = " + str(dancerLen) + "**********")
                print("******* clip name =      " + clip + " ******\n") 
                
                if(currentKeypoints.shape[0] != dancerLen):
                    timeStep = 0
                    print("****** changed dancers    = " + str(currentKeypoints.shape[0]) +  "*******")
                    print("****** Changed clip name = " + clip + "=> "+ frame + " ***********\n") 
                    for j in range(dancerLen):
                        dancerII = [] 
                        #print(len(curentFrameSequenceKeypointList))
                        for k in range(len(curentFrameSequenceKeypointList)):

                            currentStep = np.copy(curentFrameSequenceKeypointList[k][j])
                            currentStep = currentStep.flatten()
                            currentStepT = np.append(currentStep,k)
                            currentStepL = np.append(currentStepT,label)
                            dancerII.append(currentStepL)
                        
                        df = pd.DataFrame(dancerII)
                        df.to_csv("trainX.csv", index=False, mode='a', header=False)
                        print("****** saved time sequenec    = " + str(len(curentFrameSequenceKeypointList)) +  "*******")
                        print("****** saved # of dancers = " + str(dancerLen) + "dancer :"+ str(j) +  "*******\n") 
                        dancerII.clear()
                    curentFrameSequenceKeypointList.clear()
                else:
                    #stroe Current keypairs   
                    curentFrameSequenceKeypointList.append(currentKeypoints)
                    timeStep+=1
            
            if(not multiPerson):
                dancerII = [] 
                #print(len(curentFrameSequenceKeypointList))
                for k in range(len(curentFrameSequenceKeypointList)):
                            
                    currentStep = np.copy(curentFrameSequenceKeypointList[k])
                    currentStep = currentStep.flatten()
                    currentStepT = np.append(currentStep,k)
                    currentStepL = np.append(currentStepT,label)
                    dancerII.append(currentStepL)
                 
                df = pd.DataFrame(dancerII)
                df.to_csv("trainX2.csv", index=False, mode='a', header=False)
                dancerII = []
                print("****** "+clip+" saved time sequenec    = " + str(len(curentFrameSequenceKeypointList)) +  "*******")
                curentFrameSequenceKeypointList.clear()          
                continue
            
            for j in range(dancerLen):
                dancerII = [] 
                #print(len(curentFrameSequenceKeypointList))
                for k in range(len(curentFrameSequenceKeypointList)):
                            
                    currentStep = np.copy(curentFrameSequenceKeypointList[k][j])
                    currentStep = currentStep.flatten()
                    currentStepT = np.append(currentStep,k)
                    currentStepL = np.append(currentStepT,label)
                    dancerII.append(currentStepL)
                 
                df = pd.DataFrame(dancerII)
                df.to_csv("trainX.csv", index=False, mode='a', header=False)
                dancerII = []
                print("****** saved time sequenec    = " + str(len(curentFrameSequenceKeypointList)) +  "*******")
                print("****** saved # of dancers = " + str(currentKeypoints.shape[0])+ "dancer :"+ str(j)+  "*******\n") 
            curentFrameSequenceKeypointList.clear()          


#adding joint sequence to csv file for dataset
def prepareFileDataset(keypoints,step):
    
    return

In [20]:
data2 = passFrameSequence(False)

****** current frame being processed:  0.jpg ******



In [ ]:
data.flatten()

In [21]:
data2

[array([[306,  78],
        [292, 109],
        [208, 101],
        [153, 140],
        [180, 180],
        [375, 109],
        [417, 140],
        [417, 180],
        [250, 187],
        [250, 250],
        [264, 320],
        [347, 180],
        [361, 250],
        [347, 320],
        [292,  70],
        [320,  70],
        [250,  78],
        [333,  70]])]